In [1]:
#Python packages
import numpy as np
import pandas as pd
import re
from scipy import stats
import json

#Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#modeling packages
from collections import defaultdict
import lightgbm as lgb
import copy
import xgboost as xgb
import catboost as cat
from time import time
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
# from bayes_opt import BayesianOptimization
# import eli5
# import shap
# from IPython.display import HTML
from category_encoders.ordinal import OrdinalEncoder
import time
from numba import jit
import eli5
import shap
from IPython.display import HTML
import altair as alt
from category_encoders.ordinal import OrdinalEncoder
import networkx as nx

import warnings
warnings.filterwarnings('ignore')

import gc

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)

Using TensorFlow backend.
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\16507\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [19]:
train = pd.read_pickle('train_reduced.pkl')
test = pd.read_pickle('test_reduced.pkl')

print(train.shape)
print(test.shape)

(12497456, 20)
(1156413, 31)


In [22]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,month,dayofyear,Phase_Of_Day,time_by_phase_type,practice_sec,months_played,recent_ratio,total_game_time,difficulty
0,27253bdc,45bb1e1b6b50c07b,2019-09-06 17:53:46.937000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0.0,Welcome to Lost Lagoon!,Clip,NONE,9,249,Evening,0.00,47804.47,1.0,0.75,1357.0,0.0
1,27253bdc,17eeb7f223665f53,2019-09-06 17:54:17.519000+00:00,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0.0,Magma Peak - Level 1,Clip,MAGMAPEAK,9,249,Evening,0.00,47804.47,1.0,0.75,1357.0,0.0
2,77261ab5,0848ef14a8dc6892,2019-09-06 17:54:56.302000+00:00,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0.0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,9,249,Evening,203527.66,47804.47,1.0,0.75,1357.0,1.0
3,b2dba42b,0848ef14a8dc6892,2019-09-06 17:54:56.387000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53.0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,9,249,Evening,203527.66,47804.47,1.0,0.75,1357.0,1.0
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06 17:55:03.253000+00:00,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972.0,Sandcastle Builder (Activity),Activity,MAGMAPEAK,9,249,Evening,203527.66,47804.47,1.0,0.75,1357.0,1.0


In [24]:
grouped = train.groupby('installation_id',as_index=False).agg(lambda x: set(x))
grouped.head()

,installation_id,event_id,game_session,timestamp,event_data,event_count,event_code,game_time,title,type,world,month,dayofyear,Phase_Of_Day,time_by_phase_type,practice_sec,months_played,recent_ratio,total_game_time,difficulty
0,0001e90f,"{4d911100, 26fd2d99, c58186bf, b2dba42b, 5a848...","{a1ec58f109218255, 07bacda7f9437b38, 0848ef14a...","{2019-09-06 18:09:13.272000+00:00, 2019-09-06 ...","{{""coordinates"":{""x"":612,""y"":267,""stage_width""...","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{2050, 2060, 2070, 2080, 2083, 3110, 4010, 312...","{0.0, 122880.0, 178184.0, 116746.0, 36875.0, 3...","{Tree Top City - Level 1, Sandcastle Builder (...","{Activity, Clip, Game}","{MAGMAPEAK, TREETOPCITY, NONE}",{9},{249},{Evening},"{0.0, 254881.44, 203527.66}",{47804.47},{1.0},{0.75},{1357.0},"{0.0, 1.0, 2.0}"
1,000447c4,"{4d911100, f50fc6c1, a6d66e51, 4d6737eb, c952e...","{07c3b22b2de3ce4d, ae1944d61cbae05e, 29e13e528...","{2019-10-11 15:02:02.404000+00:00, 2019-10-11 ...","{{""coordinates"":{""x"":247,""y"":38,""stage_width"":...","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{5000, 2060, 5010, 2070, 3110, 4010, 3121, 402...","{0.0, 99842.0, 57348.0, 101894.0, 158729.0, 11...","{Watering Hole (Activity), Welcome to Lost Lag...","{Activity, Clip, Game}","{MAGMAPEAK, NONE}",{10},{284},{Evening},"{0.0, 254881.44, 203527.66}",{19717.938000000002},{1.0},{0.9},{181.0},"{0.0, 1.0, 2.0}"
2,0006a69f,"{f6947f54, 4d911100, 2dc29e21, bbfe0445, c5818...","{bbda08fee616189a, 80d34a30c2998653, 380481a1b...","{2019-08-06 05:36:40.855000+00:00, 2019-08-06 ...","{{""correct"":true,""caterpillars"":[4,8,5],""event...","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{2050, 4100, 4230, 5000, 4235, 2060, 4110, 501...","{0.0, 98307.0, 90116.0, 16389.0, 32774.0, 1639...","{Treasure Map, Bubble Bath, Magma Peak - Level...","{Assessment, Activity, Clip, Game}","{MAGMAPEAK, TREETOPCITY, NONE}",{8},"{241, 218, 221}","{Evening, Night}","{0.0, 254881.44, 58147.695, 203527.66, 199051....",{25980.172000000002},{1.0},{0.5},{3800.0},"{0.0, 1.0, 2.0, 3.0}"
3,0006c192,"{00c73085, f6947f54, bbfe0445, c58186bf, 8d84f...","{b6df5bbfc96a77fe, 534d791e8755d081, 5f4c5e8c9...","{2019-09-14 15:35:54.913000+00:00, 2019-10-01 ...","{{""buglength"":2,""bug"":""rollyPolly"",""duration"":...","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{2050, 4100, 4230, 4235, 2060, 4110, 2070, 208...","{0.0, 8195.0, 24596.0, 114708.0, 24611.0, 8234...","{Cauldron Filler (Assessment), Treasure Map, B...","{Assessment, Activity, Clip, Game}","{CRYSTALCAVES, MAGMAPEAK, TREETOPCITY, NONE}","{9, 10}","{256, 257, 274, 263}","{Evening, Night}","{0.0, 254881.44, 58147.695, 203527.66, 60455.1...",{37441.81200000001},{2.0},"{0.2979, -999.0}",{2224.0},"{0.0, 1.0, 2.0, 3.0}"
4,0009a5a9,"{00c73085, f50fc6c1, c58186bf, a6d66e51, b2dba...","{6cf048c5e55a969f, d78d2fa8766987f0, 6eea7d0a0...","{2019-09-11 18:41:10.417000+00:00, 2019-09-11 ...","{{""description"":""Good job, team!"",""identifier""...","{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{5000, 2060, 5010, 2070, 3110, 4010, 3120, 312...","{0.0, 43015.0, 53268.0, 90135.0, 51223.0, 2461...","{Sandcastle Builder (Activity), Slop Problem, ...","{Activity, Clip, Game}","{MAGMAPEAK, NONE}",{9},{254},"{Evening, Night}","{0.0, 254881.44, 199051.25, 203527.66}",{129544.62},{1.0},{0.75},{412.0},"{0.0, 1.0, 2.0}"
